In [1]:
import numpy as np
import scipy.special as sp

In [29]:
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        
        #初始化网络，设置输入、隐藏、输出节点数，以及学习率
        #初始化网络，设置输入层，中间层，和输出层节点数
        self.inodes = input_nodes
        self.hiddennodes = hidden_nodes
        self.onodes = output_nodes

        #设置学习率
        self.learning_rate = learning_rate
        pass
        """ 
         初始化权重矩阵，我们有两个权重矩阵,
         一个是wih表示输入层和中间层节点间链路权重形成的矩阵
         一个是who,表示中间层和输出层间链路权重形成的矩阵
         """
        #初始化权重矩阵,减去0.05是，为了初始化矩阵有正有负，拟合速度更快
          
        self.wih = (np.random.normal(0.0, pow(self.hiddennodes,-0.5), (self.hiddennodes,self.inodes) )  )
        self.who = (np.random.normal(0.0, pow(self.onodes,-0.5), (self.onodes,self.hiddennodes) )  )

    
        """ 
        scipy.special.expit(x)函数的作用是计算sigmoid函数,即：
        f(x) = 1 / (1 + exp(-x))
        它是sigmoid函数的指数形式,因此可以用作激活函数。
        它是scipy包中的一个函数,可以直接调用。
      
        """
        self.activation_function = lambda x : sp.expit(x)
        pass
    
    def train(self,inputs_list, targets_list):
        # 根据输入的训练数据更新节点 链路权重
        #根据输入的训练数据更新节点链路权重
        '''
        把inputs_list, targets_list转换成np支持的二维矩阵
        .T表示做矩阵的转置
        '''
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        #计算信号经过输入层后产生的信号量
        hidden_inputs = np.dot(self.wih, inputs)
        #中间层神经元对输入的信号做激活函数后得到输出信号
        hidden_outputs = self.activation_function(hidden_inputs)
        #输出层接收来自中间层的信号量
        final_inputs = np.dot(self.who, hidden_outputs)
        #输出层对信号量进行激活函数后得到最终输出信号
        final_outputs = self.activation_function(final_inputs)

        #计算误差
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors*final_outputs*(1-final_outputs))
        #根据误差计算链路权重的更新量，然后把更新加到原来链路权重上
        self.who += self.learning_rate * np.dot((output_errors * final_outputs *(1 - final_outputs)),
                                       np.transpose(hidden_outputs))
        self.wih += self.learning_rate * np.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)),
                                       np.transpose(inputs))
                                       
        pass

    def  query(self,inputs):
        #根据输入数据计算并输出答案
        #计算中间层从输入层接收到的信号量
        hidden_inputs = np.dot(self.wih, inputs)
        #计算中间层经过激活函数后形成的输出信号量
        hidden_outputs = self.activation_function(hidden_inputs)
        #计算最外层接收到的信号量
        final_inputs = np.dot(self.who, hidden_outputs)
        #计算最外层神经元经过激活函数后输出的信号量
        final_outputs = self.activation_function(final_inputs)
        print(final_outputs)
        return final_outputs
 
    

In [30]:
#初始化网络
'''
由于一张图片总共有28*28 = 784个数值, 因此我们需要让网络的输入层具备784个输入节点
'''
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
learning_rate = 0.1
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

#读入训练数据
#open函数里的路径根据数据存储的路径来设定
training_data_file = open("dataset/mnist_train.csv",'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [32]:
#加入epocs,设定网络的训练循环次数
epochs = 5
for e in range(epochs):
    #把数据依靠','区分，并分别读入
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]))/255.0 * 0.99 + 0.01
        #设置图片与数值的对应关系
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)

In [33]:
test_data_file = open("dataset/mnist_test.csv")
test_data_list = test_data_file.readlines()
test_data_file.close()
scores = []
for record in test_data_list:
    all_values = record.split(',')
    correct_number = int(all_values[0])
    print("该图片对应的数字为:",correct_number)
    #预处理数字图片
    inputs = (np.asfarray(all_values[1:])) / 255.0 * 0.99 + 0.01
    #让网络判断图片对应的数字
    outputs = n.query(inputs)
    #找到数值最大的神经元对应的编号
    label = np.argmax(outputs)
    print("网络认为图片的数字是：", label)
    if label == correct_number:
        scores.append(1)
    else:
        scores.append(0)
print(scores)

#计算图片判断的成功率
scores_array = np.asarray(scores)
print("perfermance = ", scores_array.sum() / scores_array.size)

该图片对应的数字为: 7
[0.05387105 0.00524872 0.07728929 0.06083007 0.01716759 0.02475824
 0.00445826 0.87868444 0.00948196 0.0237322 ]
网络认为图片的数字是： 7
该图片对应的数字为: 2
[0.32290229 0.02004332 0.50036012 0.05322793 0.00167889 0.10629156
 0.07525926 0.00622118 0.02903928 0.01238443]
网络认为图片的数字是： 2
该图片对应的数字为: 1
[0.01008384 0.95975269 0.02429104 0.03416704 0.03979138 0.04362467
 0.03254715 0.02508589 0.01130331 0.02745142]
网络认为图片的数字是： 1
该图片对应的数字为: 0
[0.69307317 0.0051353  0.03289323 0.06042519 0.00403355 0.00752194
 0.07564534 0.06261519 0.00578322 0.00286866]
网络认为图片的数字是： 0
该图片对应的数字为: 4
[0.01357098 0.0026563  0.05011997 0.05020745 0.83491465 0.00929719
 0.0083446  0.40369955 0.03209406 0.02868892]
网络认为图片的数字是： 4
该图片对应的数字为: 1
[0.0089615  0.96272904 0.02946731 0.03774946 0.03377975 0.03299519
 0.00814198 0.02320378 0.02160738 0.0179595 ]
网络认为图片的数字是： 1
该图片对应的数字为: 4
[0.0108782  0.00093301 0.02297535 0.05123116 0.27749614 0.06067982
 0.00999657 0.09188823 0.03491784 0.43832943]
网络认为图片的数字是： 9
该图片对应的数字为: 9
[0.0032